In [1]:
# 모든것은 분봉단위로
# 1분 단위로 시가 조회
    # 하나의 구간에서 거래된 첫번째 가격
# 0.2초 단위로 실시간 매수 여부 판단
# 서로 다른 두개의 로직으로 구현

In [1]:
import time
import threading
import queue
from collections import deque

import pyupbit
import datetime
from pyupbit import get_current_price, get_tickers, get_ohlcv, get_tick_size
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
class Producer(threading.Thread):
    def __init__(self, q):
        super().__init__()
        self.q = q
    
    def run(self):
        while True:
            price = get_current_price('KRW-REP')
            self.q.put(price)
            time.sleep(60 * 5)

In [3]:
class Consumer(threading.Thread):
    def __init__(self, q):
        super().__init__()
        self.q = q
        
        self.ticker = "KRW-REP"
        
        self.ma15 = deque(maxlen=15)
        self.ma50 = deque(maxlen=50)
        self.ma120 = deque(maxlen=120)
        
        df = get_ohlcv(self.ticker, interval="minute5")
        self.ma15.extend(df['close'])
        self.ma50.extend(df['close'])
        self.ma120.extend(df['close'])

    def run(self):
        price_curr = None
        hold_flag = False # 매수, 매도 후 매도주문이 완료되기 전까지는 매수 주문이 들어가면 안된다.
        wait_flag = False # 하나의 1분봉에서만 로직이 실행되게
        
        upbit = pyupbit.Upbit('UDgx294CT64wyrhkKJ4dbk8oKnCLaze5qtrM5e0M', 'YHbgxjlTzF4KYpwG3Kjhi1M91YpZNSep6aPFWgHO')
        cash = upbit.get_balance()
        print('보유현금', cash)
        
        i = 0
        
        while True:
            try:
                if not self.q.empty():
                    if price_curr != None:
                        self.ma15.append(price_curr)
                        self.ma50.append(price_curr)
                        self.ma120.append(price_curr)

                    curr_ma15 = sum(self.ma15) / len(self.ma15)
                    curr_ma50 = sum(self.ma50) / len(self.ma50)
                    curr_ma120 = sum(self.ma120) / len(self.ma120)

                    price_open = self.q.get()
                    if hold_flag == False:
                        price_buy  = price_open * 1.01
                        price_sell = price_open * 1.02
                    wait_flag = False

                    print('open: ', price_open, 'buy:', price_buy, 'price_sell:', price_sell)

                price_curr = get_current_price(self.ticker)
                if price_curr == None:
                    continue

                # 매수 조건 탐색
                if hold_flag == False and wait_flag == False and \
                    price_curr >= price_buy and \
                    curr_ma15 >= curr_ma50 and \
                    curr_ma15 <= curr_ma50 * 1.03 and \
                    curr_ma120 <= curr_ma50 :
                    # 수수료 0.05% 0.0005
                    # 매수 가능
                    # 매수 주문 후, 주문 처리 될 떄 까지 대기
                    print('매수 로직 실행')

                    buyRes = upbit.buy_market_order(self.ticker, cash * 0.9995)
                    print('매수 주문', buyRes)
                    if buyRes == None or "error" in buyRes:
                        print("매수 주문 이상")
                        continue

                    while True:
                        buyOrder = upbit.get_order(buyRes['uuid'])
                        if buyOrder != None and len(buyOrder['trades']) > 0:
                            print('매수 주문 처리 완료', buyOrder)
                            break
                        else:
                            print('매수 주문 대기중')
                            time.sleep(0.5)

                    while True:
                        volume = upbit.get_balance(self.ticker)
                        if volume != None:
                            break
                        time.sleep(0.5)

                    print('매도 로직 실행')
                    while True:
                        price_sell = get_tick_size(price_sell)
                        sellRes = upbit.sell_limit_order(self.ticker, price_sell, volume)

                        if sellRes == None or "error" in sellRes:
                            print("매도 주문 이상", sellRes)
                            time.sleep(0.5)
                        else:
                            print('매도 주문', sellRes)
                            hold_flag = True
                            break

                # 매도 주문이 체결 됐는지
                if hold_flag == True:
                    uncomp = upbit.get_order(self.ticker)
                    if uncomp != None and len(uncomp) == 0:
                        cash = upbit.get_balance()
                        if cash == None:
                            continue

                        print('매도 완료', cash)
                        hold_flag = False
                        wait_flag = True

                if i == (5 * 60 * 1):
                    print(f"[{datetime.datetime.now()}] 현재가 {price_curr}, 목표가 {price_buy}, ma {curr_ma15:.2f}/{curr_ma50:.2f}/{curr_ma120:.2f}, hold_flag {hold_flag}, wait_flag {wait_flag}")
                    i = 0
                i += 1
            except:
                print("error")
                
            time.sleep(0.2)
                
            

In [4]:
q = queue.Queue()
Producer(q).start()
Consumer(q).start()

보유현금 8588.21351499
open:  20600.0 buy: 20806.0 price_sell: 21012.0
[2022-02-02 22:31:04.564983] 현재가 20590.0, 목표가 20806.0, ma 20965.33/20873.20/19950.75, hold_flag False, wait_flag False
[2022-02-02 22:32:26.327533] 현재가 20540.0, 목표가 20806.0, ma 20965.33/20873.20/19950.75, hold_flag False, wait_flag False
[2022-02-02 22:33:48.516320] 현재가 20400.0, 목표가 20806.0, ma 20965.33/20873.20/19950.75, hold_flag False, wait_flag False
open:  20460.0 buy: 20664.6 price_sell: 20869.2
[2022-02-02 22:35:10.584267] 현재가 20500.0, 목표가 20664.6, ma 20909.33/20871.40/19983.00, hold_flag False, wait_flag False
[2022-02-02 22:36:34.202061] 현재가 20540.0, 목표가 20664.6, ma 20909.33/20871.40/19983.00, hold_flag False, wait_flag False
[2022-02-02 22:37:56.869509] 현재가 20580.0, 목표가 20664.6, ma 20909.33/20871.40/19983.00, hold_flag False, wait_flag False
매수 로직 실행
매수 주문 {'uuid': 'bbde5e39-fdc3-465c-8d48-23ce7fe58ede', 'side': 'bid', 'ord_type': 'price', 'price': '8583.9194', 'state': 'wait', 'market': 'KRW-REP', 'created_at

In [7]:
def get_coin():
    tickers = get_tickers(fiat='KRW')
    tickers = ['KRW-REP']
    
    for ticker in tickers:
        ma15 = deque(maxlen=15)
        ma50 = deque(maxlen=50)
        ma120 = deque(maxlen=120)
        
        df = get_ohlcv(ticker, interval="minute5")
        ma15.extend(df['close'])
        ma50.extend(df['close'])
        ma120.extend(df['close'])
        
        curr_ma15 = sum(ma15) / len(ma15)
        curr_ma50 = sum(ma50) / len(ma50)
        curr_ma120 = sum(ma120) / len(ma120)
#         print(curr_ma15 >= curr_ma50 and curr_ma15 <= curr_ma50 * 1.03 and curr_ma120 <= curr_ma50)
        
        if curr_ma15 >= curr_ma50 and curr_ma15 <= curr_ma50 * 1.03 and curr_ma120 <= curr_ma50:
            price_curr = get_current_price(ticker)
            print(ticker, price_curr)
                
        time.sleep(0.2)
    
get_coin()

KRW-REP 20590.0


In [ ]:
upbit = pyupbit.Upbit('UDgx294CT64wyrhkKJ4dbk8oKnCLaze5qtrM5e0M', 'YHbgxjlTzF4KYpwG3Kjhi1M91YpZNSep6aPFWgHO')
volume = upbit.get_balance('KRW-ALGO')
print(volume)

In [ ]:
get_tick_size(154001.123123123123)